In [5]:
from statistics import mode
import matplotlib.pyplot as plt
import gspread as g
import numpy as np

x = [5,21,24,42,54,34,60,101,112,99]
x = np.array(x)
y = [7,22,28,52,63,90,55,122,131,199]
y = np.array(y)

gc = g.service_account(filename='unitydatascience-364706-1c1ce20c7b07.json')
sh = gc.open("UnitySheets").worksheet('Sheet2')

def model(a, b, x):
    return a * x + b

def loss_function(a, b, x, y):
    num = len(x)
    prediction = model(a, b, x)
    return (0.5 / num) * (np.square(prediction - y)).sum()

def optimize(Lr, a, b, x, y):
    num = len(x)
    prediction = model(a, b, x)
    da = (1.0 / num) * ((prediction - y) * x).sum()
    db = (1.0 / num) * ((prediction - y).sum())
    a = a - Lr * da
    b = b - Lr * db
    return a, b

def iterate(Lr, a, b, x, y, times):
    for i in range(times):
        a, b = optimize(Lr, a, b, x, y)
    return a,b  

Lr = 0.000001

a_rand = np.random.rand(1)
b_rand = np.random.rand(1)

a = np.copy(a_rand)
b = np.copy(b_rand)
iter_range = np.arange(1, 6) * 200
iter_range = np.concatenate([[1], iter_range])

a_loss = []

for iter, num in enumerate(iter_range):
    a,b = iterate(Lr, a,b,x,y, num)
    prediction = model(a,b,x)
    loss = loss_function(a, b, x, y)
    a_loss.append(loss)

    sh.update(('A' + str(iter + 1)), str(num))
    sh.update(('B' + str(iter + 1)), f"{a[0]:.3f}".replace('.',','))
    sh.update(('C' + str(iter + 1)), f"{b[0]:.3f}".replace('.',','))
    sh.update(('D' + str(iter + 1)), f"{loss:.3f}".replace('.',','))